In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

In [2]:
df=pd.read_csv('finalPSL.csv')

In [3]:
df.head()

,team_1,team_2,total_runs,wickets_left,balls_left,crr,last_five,Target
0,Quetta Gladiators,Islamabad United,27,8,100,6.230769,26.0,156
1,Lahore Qalandars,Quetta Gladiators,12,10,108,4.000000,30.0,209
2,Multan Sultans,Lahore Qalandars,1,10,116,0.600000,41.0,200
3,Lahore Qalandars,Peshawar Zalmi,92,8,43,6.650602,65.0,164
4,Karachi Kings,Lahore Qalandars,107,7,54,8.916667,46.0,186


In [4]:
df.shape

(19689, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19689 entries, 0 to 19688
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   team_1        19689 non-null  object 
 1   team_2        19689 non-null  object 
 2   total_runs    19689 non-null  int64  
 3   wickets_left  19689 non-null  int64  
 4   balls_left    19689 non-null  int64  
 5   crr           19689 non-null  float64
 6   last_five     19689 non-null  float64
 7   Target        19689 non-null  int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 1.2+ MB


In [6]:
X=df.drop(columns='Target')
Y=df['Target']
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=1)

In [7]:
X_train.shape

(16735, 7)

In [8]:
X_test.shape

(2954, 7)

### XGBOOST

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.model_selection import cross_val_score

In [10]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['team_1','team_2'])
]
,remainder='passthrough')

In [11]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=100,learning_rate=0.2,max_depth=12,random_state=1))
])

In [12]:
pipe.fit(X_train,Y_train)
y_pred = pipe.predict(X_test)
print(r2_score(Y_test,y_pred))
print(mean_absolute_error(Y_test,y_pred))

0.922534963476034
4.345068690739698


In [13]:
#OVERFITTING
training_predictions = pipe.predict(X_train)
r2_score(Y_train,training_predictions)

0.998936056422758

In [14]:
cv_scores = cross_val_score(pipe,X,Y,cv=5, scoring='r2')

print("Cross-Validation Scores:", cv_scores)
print("Mean Cross-Validation Score:", cv_scores.mean())

Cross-Validation Scores: [0.9112709  0.9106253  0.90370174 0.8904284  0.92261935]
Mean Cross-Validation Score: 0.9077291374321138


### Linear Regression

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer


In [16]:
transformer = ColumnTransformer(transformers=[
    ('trf',OneHotEncoder(sparse=False,drop='first'),['team_1','team_2'])
],remainder='passthrough')
X_train_transformed = transformer.fit_transform(X_train)

In [17]:
transformer = ColumnTransformer(transformers=[
    ('trf',OneHotEncoder(sparse=False,drop='first'),['team_1','team_2'])
],remainder='passthrough')
X_test_transformed = transformer.fit_transform(X_test)

In [18]:
Sscaler = StandardScaler()
X_train = Sscaler.fit_transform(X_train_transformed )
X_test = Sscaler.transform(X_test_transformed )


In [19]:
model = LinearRegression()
model.fit(X_train,Y_train)

LinearRegression()

In [20]:
predictions = model.predict(X_test)


In [21]:
#OVERFITTING
training_predictions = model.predict(X_train)
r2_score(Y_train,training_predictions)

0.46521263115130795

In [22]:
mse = mean_squared_error(Y_test,predictions)
r2 = r2_score(Y_test,predictions)
print('Mean Squared error', mse)
print('r2 Score', r2)

Mean Squared error 491.17038417755975
r2 Score 0.47289419024790547


In [23]:
cv_score = cross_val_score(model,X_train,Y_train,cv=5, scoring='r2')

print("Cross-Validation Scores:", cv_score)
print("Mean Cross-Validation Score:", cv_score.mean())

Cross-Validation Scores: [0.46671125 0.4659469  0.45562258 0.46964071 0.46226959]
Mean Cross-Validation Score: 0.4640382093618296


### XGBoost 2

In [24]:
model2 = XGBRegressor(n_estimators=200,learning_rate=0.2,max_depth=12,random_state=1)
model2.fit(X_train,Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=1, ...)

In [25]:
predictions2 = model2.predict(X_test)

In [26]:
#OVERFITTING
training_predictions = model2.predict(X_train)
r2_score(Y_train,training_predictions)

0.9996288531587854

In [27]:
mae = mean_absolute_error(Y_test,predictions2)
r2 = r2_score(Y_test,predictions2)
print('Mean Absolute Error ', mae)
print('r2 Score ',r2)

Mean Absolute Error  4.265541398210467
r2 Score  0.9234153165168264


In [28]:
cv_score = cross_val_score(model2,X_train,Y_train,cv=5, scoring='r2')

print("Cross-Validation Scores:", cv_score)
print("Mean Cross-Validation Score:", cv_score.mean())

Cross-Validation Scores: [0.89302079 0.8735891  0.88694553 0.89008585 0.8898747 ]
Mean Cross-Validation Score: 0.8867031961057105


### Decision tree

In [29]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [30]:
model3 = DecisionTreeRegressor(random_state=42)
model3.fit(X_train,Y_train)

DecisionTreeRegressor(random_state=42)

In [31]:
predictions3 = model3.predict(X_test)

In [32]:
#OVERFITTING
training_predictions = model3.predict(X_train)
r2_score(Y_train,training_predictions)

0.999667696287048

In [33]:
mse = mean_squared_error(Y_test,predictions3)
r2 = r2_score(Y_test,predictions3)
print('Mean Squared Error : ', mse)
print('r2 Score : ',r2)

Mean Squared Error :  157.38887948544345
r2 Score :  0.8310961013945031


In [34]:
cv_score = cross_val_score(model3,X_train,Y_train,cv=5, scoring='r2')

print("Cross-Validation Scores:", cv_score)
print("Mean Cross-Validation Score:", cv_score.mean())

Cross-Validation Scores: [0.77179351 0.77848067 0.77142015 0.77553906 0.77150539]
Mean Cross-Validation Score: 0.7737477562581871


### Random Forest

In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

In [36]:
model4 = RandomForestRegressor(n_estimators=100, random_state=42)
model4.fit(X_train, Y_train)

RandomForestRegressor(random_state=42)

In [37]:
prediction4 = model4.predict(X_test)

In [38]:
#OVERFITTING
training_predictions = model4.predict(X_train)
r2_score(Y_train,training_predictions)

0.9864905588331206

In [39]:
mse = mean_squared_error(Y_test,prediction4)
r2 = r2_score(Y_test,prediction4)
print('Mean Squared Error : ', mse)
print('r2 Score : ', r2)

Mean Squared Error :  74.31076815459782
r2 Score :  0.9202524441960868


In [40]:
cv_score = cross_val_score(model4,X_train,Y_train,cv=5, scoring='r2')

print("Cross-Validation Scores:", cv_score)
print("Mean Cross-Validation Score:", cv_score.mean())

Cross-Validation Scores: [0.88927513 0.87985259 0.88549266 0.89537206 0.88665289]
Mean Cross-Validation Score: 0.8873290687841227


### SVM(svr)

In [41]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [42]:
model5 = SVR(kernel='linear', C=1.0)
model5.fit(X_train, Y_train)

SVR(kernel='linear')

In [43]:
predictions5 = model5.predict(X_test)

In [44]:
mse = mean_squared_error(Y_test,predictions5)
r2 = r2_score(Y_test,predictions5)
print('Mean Squared Error : ', mse)
print('r2 Score : ', r2)

Mean Squared Error :  504.4354803565474
r2 Score :  0.4586585818152581


In [46]:
#OVERFITTING
training_predictions = model5.predict(X_train)
r2_score(Y_train,training_predictions)

0.44864343229376014

In [45]:
cv_score = cross_val_score(model5,X_train,Y_train,cv=5, scoring='r2')

print("Cross-Validation Scores:", cv_score)
print("Mean Cross-Validation Score:", cv_score.mean())

Cross-Validation Scores: [0.45517102 0.44549237 0.43661593 0.45590752 0.44647614]
Mean Cross-Validation Score: 0.4479325951616602
